In [7]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.recommandation_db

In [3]:
prove_set=db.ratings.aggregate([
    {
        '$match':{'rating':{'$gte':4.5}}
    },
    {
        '$match':{
            '$expr':{
                '$lt':[0.8,{'$rand':{}}]
            }
        }
    },
    {
        '$out':'prove_set'
    }
])

In [2]:
db.command('collstats','prove_set')['count']

1163719

In [3]:
train_set=db.ratings.aggregate([
    {
        '$match':{'rating':{'$gte':4.5}}
    },
    {
        '$lookup':
        {
            'from':'prove_set',
            'localField':'_id',
            'foreignField':'_id',
            'as':'prove_set'
        },
    },
    {
        '$project': 
        { 
            'userId': 1, 
            'movieId': 1,
            'rating': 1,
            'timestamp': 1,
            'isAnyTrue': {'$anyElementTrue': ["$prove_set"]}, 
        }
    },
    {
        '$match':{'isAnyTrue':False}
    },
    {
        '$out':'train_set'
    }
])

In [4]:
db.command('collstats','train_set')['count']

4649294

In [5]:
db.command('collstats','prove_set')['count']

1163719